# setting
- 전문 서점 부분 제외 ver

In [1]:
# 필수 라이브러리 로딩
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver import ActionChains # 스크롤
from urllib import request # 이미지
import csv
import time
# page로딩과 구조 로딩에 gab이 있어서 원하는 tag나올떄까지 기다리는거
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

import numpy as np # 결측값 삽입용
import re # 정규표현식
import pandas as pd

In [2]:
## 교재 테이블 리스트들 
main_title = []  # 본제
sub_title = []  # 부제
author = []  # 저자
publish = []  # 출판
date = []  # 발행일
cate = []  # 네이버 카테고리
tot_page = []  # 전체 쪽수
size = []  # 크기
wei = []  # 무게
img = [] # 이미지
book_info = []  # 책소개
book_review = []  # 책서평
kobo_cate = []
title_isbn = [] # isbn 교재명
set_isbn = [] #set
isbn = []
book_type = [] #제본
ver = []  # 판정보
last_ver = []  # 이전 판 정보
pn_isbn = [] # 페이지 관련 끌기
keyword = []  # 키워드
price = []

index = []
idx_page = []

# 관련 사전 정의
info_dic = {'author':author, 'publish': publish, 'date':date}
isbn_dic = {'isbn':isbn, 'book_type': book_type, 'ver':ver,'last_ver':last_ver ,'pn_isbn':pn_isbn ,\
           'keyword':keyword, 'set_isbn': set_isbn, 'price':price}

# 함수 정의 

In [3]:
def naver():
    # 창 전환
    driver.switch_to.window(driver.window_handles[1]) 
    driver.implicitly_wait(5)
    try: # 제목 수집 -부제가 없는지 확인먼저 (이게 밑에 가니깐 제목이 두번 들어감)
        sub_title.append(driver.find_element(By.CLASS_NAME,f'bookTitle_sub_title__B0uMS').text) # 부
        main_title.append(driver.find_element(By.CLASS_NAME,f'bookTitle_book_name__JuBQ2').text) # 본
        
    except:
        main_title.append(driver.find_element(By.CLASS_NAME,f'bookTitle_title_area__fspvB').text) #부제랑 안나눠졌을시
        sub_title.append(np.NaN)
    # 저자, 출판, 발행일 수집
    driver.implicitly_wait(5)
    try:
        info_func()
    except:
        for x in [author, publish, date]:
            x.append(np.NaN)

    try:
        code = sub1_func()
    except:
        print('sub1함수 예외상황')
    
    try:
        sub2_func()
    except:
        print('sub2함수 예외상황')
    
    try:
        img_func(code)
    except:
        img.append(np.nan)
    # 소개
    try:
        text = driver.find_element(By.XPATH, '//*[@id="book_section-info"]/div[3]/div[2]/div[1]').text.split('\n')
        text = [t for t in text if t != '' ]
        book_info.append(text)
    except:
        book_info.append(np.NaN)
        
    idx_func() 
    return code

In [4]:
def info_func():
    info = driver.find_elements(By.CLASS_NAME, "bookTitle_item_info__IMLyY")
    check = []
    for i in range(len(info)):
        a = info[i].text.split('\n')
        if a[0] == '저자':
            check.append('author')
            author.append(a[1]) # 저자
        elif a[0] == '출판':
            check.append('publish')
            publish.append(a[1])
        elif a[0] == '발행':
            check.append('date')
            date.append(a[1])
        else:
            pass
    for c in ['author','publish', 'date']:
        if c not in check:
            info_dic[c].append(np.NaN)
    
def sub1_func():
    sub1 = driver.find_elements(By.CLASS_NAME, "bookBasicInfo_info_detail__I0Fx5") # 카테고리, ISBN
    # cate
    cate.append(sub1[0].text)
    # code
    code = sub1[2].text
    return code

def sub2_func():
    sub2 = driver.find_elements(By.CLASS_NAME, "bookBasicInfo_spec__qmQ_N") # 쪽수, 크기(무게)
    if len(sub2) == 3:
        tot_page.append(sub2[0].text)
        # wei
        wei.append(sub2[1].text)
        # size
        size.append(sub2[2].text)
    elif len(sub2) == 2:
        tot_page.append(sub2[0].text)
        # wei
        wei.append(np.NaN)
        # size
        size.append(sub2[1].text)
    elif len(sub2) == 1:
        tot_page.append(sub2[0].text)
        wei.append(np.NaN)
        size.append(np.NaN)
    else:
        tot_page.append(np.NaN)
        wei.append(np.NaN)
        size.append(np.NaN)
        
def img_func(code):
    # image
    img_url = driver.find_element(By.XPATH, '//*[@id="container"]/div[2]/div[1]/div[1]/div[1]/div[1]/div/img').get_attribute("src")
    driver.implicitly_wait(5)
    request.urlretrieve(img_url,f"C:\\police_img\\{code}.jpg")
    img.append(f'{code}.jpg')
    
# index 함수 정의
def page(text,ls):
    t_ls = []
    for t in txts:
        x = t.split(' ')
        try:
            if x[-1].isnumeric() == True:
                t_ls.append(x[-1]) # sub_p
            else:
                t_ls.append(np.NaN)
        except:
            t_ls.append(np.NaN)   
    ls.append(t_ls)
    
def idx_func():
    try: 
        scroll = driver.find_element(By.PARTIAL_LINK_TEXT,'목차')
    # somthing element까지 스크롤
        act = ActionChains(driver)  # 수정한 부분
        act.move_to_element(scroll).perform()
        scroll.click()     # 수정한 부분
        driver.implicitly_wait(5)
        ex = driver.find_elements(By.CLASS_NAME,'infoItem_data_text__bUgVI')
        txts = ex[-1].text.split('\n')
        txts = [t for t in txts if t != '']
        page(txts, idx_page)
        index.append(txts)
    except:
        index.append(np.NaN)
        idx_page.append(np.NaN)

- ISBN은 세트랑 세트가 아닌것이 불리되어야 하니깐, 해당페이지 들어가자 마자
resultData의 개수를 카운트해서 하나면 단일 하나 이상이면 세트 권으로 파악을 하자
re.sub(r'[^0-9]', '', string)

In [5]:
def one_isbn():
    # 가격
    check = []
#     pr = driver.find_element(By.XPATH, '//*[@id="resultList_div"]/form/div/div/div/ul/li[6]').text
#     price.append(pr.split(' ')[-1])
#     check.append('price')
    driver.find_element(By.XPATH, f'//*[@id="resultList_div"]/form/div[{1}]/div/div/div/a').click()
    driver.implicitly_wait(5)
    tit = driver.find_element(By.CSS_SELECTOR,'#contents > div > div.resultViewDetail > div.tit').text
    title_isbn.append(tit.split(']')[1])
    #     num = driver.find_elements(By.XPATH, '//*[@id="contents"]/div/div[1]/div[2]/div[2]/ul/li/div')
    num = driver.find_elements(By.XPATH, '//*[@id="contents"]/div/div[1]/div[2]/div[2]/ul/li')
    
    for n in range(len(num)):
        a = num[n].text.split('\n')
        if a[0] == 'ISBN':
            check.append('isbn')
            isbn.append(re.sub(r'[^0-9]', '', a[1].split(' ')[0])) # 저자
        elif a[0] in ['세트ISBN','SET ISBN']:
            check.append('set_isbn')
            set_isbn.append(a[1])
        elif a[0] in ['제본형식','제본형식(종이)']:
            check.append('book_type')
            book_type.append(a[1])
        elif a[0] == '판사항':
            check.append('ver')
            ver.append(a[1])
        elif a[0] == '이전판 정보':
            check.append('last_ver')
            last_ver.append(re.sub(r'[^0-9]', '', a[1]))
        elif a[0] == '형태 및 본문언어':
            check.append('pn_isbn')
            pn_isbn.append(a[1])
        elif a[0] == '키워드':
            check.append('keyword')
            keyword.append(a[1])
        elif a[0] in ['가격', '가격정보']:
            if re.sub(r'[^0-9]', '', a[1]) != '':
                price.append(re.sub(r'[^0-9]', '', a[1]))
                check.append('keyword')
            else:
                price.append(np.NaN)
        else:
            pass

    for c in ['isbn','book_type','ver','last_ver','pn_isbn','keyword', 'set_isbn','price']:
        if c not in check:
            isbn_dic[c].append(np.NaN)
        else:
            pass
    driver.close() # 종료
    driver.implicitly_wait(5)
    driver.switch_to.window(driver.window_handles[0])


In [6]:
main_title,sub_title,author,publish ,date ,cate,tot_page ,size ,wei ,img ,book_info,book_review 

([], [], [], [], [], [], [], [], [], [], [], [])

In [7]:
def set_code():
    # 가격
    prs = driver.find_elements(By.CLASS_NAME, 'dot-list')
    for i in range(len(prs), 0, -1):
        # 상품 선택
        driver.find_element(By.XPATH, f'//*[@id="resultList_div"]/form/div[{i}]/div/div/div/a').click()
        driver.implicitly_wait(5)
        tit = driver.find_element(By.CSS_SELECTOR,'#contents > div > div.resultViewDetail > div.tit').text
        title_isbn.append(tit.split(']')[1]) # title 크롤링
        # 기존 교재 정보 널값 처리
        for j in [main_title,sub_title,author,publish,date,cate,tot_page,size,wei,img,book_info,book_review]:
            j.append(np.NaN)
        num = driver.find_elements(By.XPATH, '//*[@id="contents"]/div/div[1]/div[2]/div[2]/ul/li')
        check = []
        for n in range(len(num)):
            a = num[n].text.split('\n')
            if a[0] == 'ISBN':
                check.append('isbn')
                isbn.append(re.sub(r'[^0-9]', '', a[1].split(' ')[0])) 
            elif a[0] in ['세트ISBN','SET ISBN']:
                check.append('set_isbn')
                set_isbn.append(re.sub(r'[^0-9]', '', a[1].split(' ')[0]))
            elif a[0] in ['제본형식','제본형식(종이)'] :
                check.append('book_type')
                book_type.append(a[1])
            elif a[0] == '판사항':
                check.append('ver')
                ver.append(a[1])
            elif a[0] == '이전판 정보':
                check.append('last_ver')
                last_ver.append(re.sub(r'[^0-9]', '', a[1]))
            elif a[0] == '형태 및 본문언어':
                check.append('pn_isbn')
                pn_isbn.append(a[1])
            elif a[0] == '키워드':
                check.append('keyword')
                keyword.append(a[1])
            elif a[0] in ['가격', '가격정보']:
                if re.sub(r'[^0-9]', '', a[1]) != '':
                    price.append(re.sub(r'[^0-9]', '', a[1]))
                    check.append('keyword')
                else:
                    price.append(np.NaN)
            else:
                pass

        for c in ['isbn','book_type','ver','last_ver','pn_isbn','keyword', 'set_isbn','price']:
            if c not in check:
                isbn_dic[c].append(np.NaN)
        driver.back()
        driver.implicitly_wait(5)
    driver.close() # 종료
    driver.switch_to.window(driver.window_handles[0])

In [8]:
def search_isbn(code): 
    driver.get(f'https://www.nl.go.kr/seoji/contents/S80100000000.do?schType=simple&schStr={code}')
    driver.implicitly_wait(5)
    num = driver.find_elements(By.CLASS_NAME, 'resultData')
    if len(num) == 1: # 단일 경우 함수
        one_isbn()
    else:
        set_code()

In [9]:
def find_new():
    try:
        elements = driver.find_elements(By.CLASS_NAME,f'bookListItem_vendor__d_xBD')
        print(len(elements),"상품수")
        for i,ele in enumerate(elements):
            print(i, ele.text)
            ele.click()
            driver.implicitly_wait(5)
            print('상품클릭')
            # 네이버 함수
            isbn = naver()
            print(isbn)
            search_isbn(isbn)
            print('서치')
    except:
        print("find예외")
        driver.back()
        driver.implicitly_wait(5)

# driver세팅 및 수집

In [10]:
# 첫화면
URL = 'https://search.shopping.naver.com/book/search/category?bookTabType=ALL&catId=50005981&pageIndex=1&pageSize=40&sort=DATE'
ops = webdriver.ChromeOptions()
ops.add_experimental_option('excludeSwitches', ['enable-logging'])

sv = Service(executable_path= 'C:\Data\chromedriver')
driver = webdriver.Chrome(service=sv, options=ops)

driver.get(url=URL)

In [11]:
tag = driver.find_element(By.CLASS_NAME,'style_footer__DaTQt')
# somthing element까지 스크롤
action = ActionChains(driver)
moment = WebDriverWait(driver, 6).until(EC.presence_of_element_located((By.CLASS_NAME, 'style_footer__DaTQt')))
for n in range(2,50):
    print(f'{n-1}페이지')
    action.move_to_element(tag).perform()
    moment
    find_new()
    moment
    print('for문 페이지 클릭전', n)
    driver.implicitly_wait(5)
    driver.find_element(By.LINK_TEXT,f'{n}').send_keys(Keys.ENTER)
        #driver.implicitly_wait(5)
    print('for문 페이지 클릭후')

1페이지
4 상품수
0 판매처 43
상품클릭
9791166184604
서치
1 판매처 16
상품클릭
9791197865367
서치
2 판매처 27
상품클릭
9791167930927
서치
3 판매처 36
상품클릭
9788952645425
서치
for문 페이지 클릭전 2
for문 페이지 클릭후
2페이지
2 상품수
0 판매처 25
상품클릭
9791192818160
서치
1 판매처 14
상품클릭
9791165234355
서치
for문 페이지 클릭전 3
for문 페이지 클릭후
3페이지
1 상품수
0 판매처 20
상품클릭
9791163485810
서치
for문 페이지 클릭전 4
for문 페이지 클릭후
4페이지
2 상품수
0 판매처 29
상품클릭
9791165234331
서치
1 판매처 35
상품클릭
9791198342201
서치
for문 페이지 클릭전 5
for문 페이지 클릭후
5페이지
0 상품수
for문 페이지 클릭전 6
for문 페이지 클릭후
6페이지
0 상품수
for문 페이지 클릭전 7
for문 페이지 클릭후
7페이지
3 상품수
0 판매처 20
상품클릭
9791165234348
서치
1 판매처 20
상품클릭
9791165234317
서치
2 판매처 33
상품클릭
9791160492699
서치
for문 페이지 클릭전 8
for문 페이지 클릭후
8페이지
0 상품수
for문 페이지 클릭전 9
for문 페이지 클릭후
9페이지
1 상품수
0 판매처 30
상품클릭
9791162933916
서치
for문 페이지 클릭전 10
for문 페이지 클릭후
10페이지
0 상품수
for문 페이지 클릭전 11
for문 페이지 클릭후
11페이지
0 상품수
for문 페이지 클릭전 12
for문 페이지 클릭후
12페이지
1 상품수
0 판매처 39
상품클릭
9791169660488
서치
for문 페이지 클릭전 13
for문 페이지 클릭후
13페이지
0 상품수
for문 페이지 클릭전 14
for문 페이지 클릭후
14페이지
0 상품수
for문 페이지 클릭전 15
for문 페이지 클릭후
15페이지
1 상

In [12]:
len(main_title),len(author),len(book_info),len(isbn),len(price),len(index), len(idx_page), len(title_isbn)

(40, 40, 40, 39, 78, 37, 37, 39)

In [14]:
test = pd.DataFrame({'main_title': main_title, 'sub_title':sub_title,'cate':cate, 'tot_page':tot_page,\
                     'author':author,'publish':publish, 'date':date, 'size':size, 'wei':wei,'img':img,\
                    'book_info':book_info})
#                      'set_isbn':set_isbn, 'isbn':isbn,\
#                      'title_isbn':title_isbn,'price':price, 'book_type':book_type, 'ver':ver,\
#                      'last_ver':last_ver,'pn_isbn':pn_isbn,'keyword':keyword})

test.to_excel('C:/police_data/ex_police_book2.xlsx',encoding = 'utf-8-sig', index=False)

C:\Data\conda\lib\site-packages\pandas\util\_decorators.py:211: FutureWarning: the 'encoding' keyword is deprecated and will be removed in a future version. Please take steps to stop the use of 'encoding'
  return func(*args, **kwargs)


In [ ]:
idx_test = pd.DataFrame({'main_title': main_title, 'isbn':isbn, 'index': index, 'idx_page':idx_page})
idx_test.to_excel('C:/police_data/ex_police_idx2.xlsx',encoding = 'utf-8-sig', index=False)

In [16]:
name_test = pd.DataFrame({'main_title': main_title, 'sub_title':sub_title,'isbn':isbn,\
                          'set_isbn':set_isbn,'title_isbn': title_isbn})
name_test.to_excel('C:/police_data/ex_police_name2.xlsx',encoding = 'utf-8-sig', index=False)

C:\Data\conda\lib\site-packages\pandas\util\_decorators.py:211: FutureWarning: the 'encoding' keyword is deprecated and will be removed in a future version. Please take steps to stop the use of 'encoding'
  return func(*args, **kwargs)


In [15]:
isbn.append(''), set_isbn.append(''), title_isbn.append('')

(None, None, None)

In [20]:
idx_page

[nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan]